环境准备
%pip install datasets langchain sentence_transformers tqdm chromadb langchain_wenxin

In [5]:
# 本地数据加载
from langchain.document_loaders import TextLoader
loader = TextLoader("C:\\Users\\administered\\ai\\deepLearning\\li.txt", encoding="utf-8")
documents = loader.load()
documents

[Document(metadata={'source': 'C:\\Users\\administered\\ai\\deepLearning\\li.txt'}, page_content='藜（读音lí）麦（Chenopodium\\xa0quinoa\\xa0Willd.）是藜科藜属植物。穗部可呈红、紫、黄，植株形状类似灰灰菜，成熟后穗部类似高粱穗。植株大小受环境及遗传因素影响较大，从0.3-3米不等，茎部质地较硬，可分枝可不分。单叶互生，叶片呈鸭掌状，叶缘分为全缘型与锯齿缘型。藜麦花两性，花序呈伞状、穗状、圆锥状，藜麦种子较小，呈小圆药片状，直径1.5-2毫米，千粒重1.4-3克。\\xa0[1]\\xa0\\n原产于南美洲安第斯山脉的哥伦比亚、厄瓜多尔、秘鲁等中高海拔山区。具有一定的耐旱、耐寒、耐盐性，生长范围约为海平面到海拔4500米左右的高原上，最适的高度为海拔3000-4000米的高原或山地地区。\\xa0[1]\\xa0\\n藜麦富含的维生素、多酚、类黄酮类、皂苷和植物甾醇类物质具有多种健康功效。')]

In [9]:
# 文档分割
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 采用固定字符长度分割chunk_size=128
# 自定义中文分隔符（按优先级排序）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=128,          # 最大字符数
    chunk_overlap=0,         # 片段重叠数
    separators=["\n\n", "\n", "。", "！", "？", "，", "；", "、"]  # 中文优先分割符
)

documents = text_splitter.split_documents(documents)
documents
print(len(documents[0].page_content),'\n', documents)  # 输出文本的字符数

120 
 [Document(metadata={'source': 'C:\\Users\\administered\\ai\\deepLearning\\li.txt'}, page_content='藜（读音lí）麦（Chenopodium\\xa0quinoa\\xa0Willd.）是藜科藜属植物。穗部可呈红、紫、黄，植株形状类似灰灰菜，成熟后穗部类似高粱穗。植株大小受环境及遗传因素影响较大，从0.3-3米不等，茎部质地较硬，可分枝可不分'), Document(metadata={'source': 'C:\\Users\\administered\\ai\\deepLearning\\li.txt'}, page_content='。单叶互生，叶片呈鸭掌状，叶缘分为全缘型与锯齿缘型。藜麦花两性，花序呈伞状、穗状、圆锥状，藜麦种子较小，呈小圆药片状，直径1.5-2毫米，千粒重1.4-3克。\\xa0[1]\\xa0\\n原产于南美洲安第斯山脉的哥伦比亚、厄瓜多尔、秘鲁等中高海拔山区'), Document(metadata={'source': 'C:\\Users\\administered\\ai\\deepLearning\\li.txt'}, page_content='。具有一定的耐旱、耐寒、耐盐性，生长范围约为海平面到海拔4500米左右的高原上，最适的高度为海拔3000-4000米的高原或山地地区。\\xa0[1]\\xa0\\n藜麦富含的维生素、多酚、类黄酮类、皂苷和植物甾醇类物质具有多种健康功效。')]


In [ ]:
# 接下来对分割后的数据进行embedding，并写入数据库。这里选用m3e-base作为embedding模型，向量数据库选用Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

model_name = "moka-ai/m3e-base"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs
    encode_kwargs = encode_kwargs
    query_instruction = "为文本生成向量表示用于文本检索"              
)

db = Chroma.from_documents(documents, embedding)
db.similarity_search('藜一般在几月播种?')

In [ ]:
# prompt设计
template = '''
        【任务描述】
        请根据用户输入的上下文回答问题，并遵守回答要求。

        【背景知识】
        {{context}}

        【回答要求】
        - 你需要严格根据背景知识的内容回答，禁止根据常识和已知信息回答问题。
        - 对于不知道的信息，直接回答“未找到相关答案”
        -----------
        {question}
        '''

这里采用ConversationalRetrievalChain，ConversationalRetrievalQA chain 是建立在 RetrievalQAChain 之上，提供历史聊天记录组件。如下面定义了memory来追踪聊天记录，在流程上，先将历史问题和当前输入问题融合为一个新的独立问题，然后再进行检索，获取问题相关知识，最后将获取的知识和生成的新问题注入Prompt让大模型生成回答。

In [10]:
from langchain import LLMChain
from langchain_wenxin.llms import Wenxin
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# llm选型
llm = Wenxin(model = 'ernie-bot', baidu_api_key = 'baidu_api_key', baidu_secret_key = 'baidu_secret_key')

retriever = db.as_retriever()
memory = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever, memory = memory)
qa({"question": "藜怎么防治虫害？"})

PydanticUserError: If you use `@root_validator` with pre=False (the default) you MUST specify `skip_on_failure=True`. Note that `@root_validator` is deprecated and should be replaced with `@model_validator`.

For further information visit https://errors.pydantic.dev/2.11/u/root-validator-pre-skip